# 신경망 모델 구성하기

신경망 데이터에 대한 연산을 수행하는 계층/모듈로 구성되어 있습니다.
torch.nn네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공합니다.
Pytorch의 모든 모듈은 nn.Module의 하위 클래스 입니다.
신경망은 다른 모듈로 구성된 모듈입니다. 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있습니다.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# 학습을 위한 장치 얻기
가능한 경우 GPU 또는 MPS와 같은 하드웨어 가속기에서 모델을 학습하려고 합니다. torch.cuda 또는 torch.backends.mps가 사용가능한지 확인해보고, 그렇지 않다면 CPU를 계속 사용합니다.

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


# 클래스 정의하기

신경망 모델을 nn.Module 의 하위 클래스로 정의하고, _init_ 에서 신경망 계층들을 초기화합니다.
nn.Module을 상속 받은 모든 클래스는 forward 메소드에 입력 데이터에 대한 연산들을 구현합니다.



In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


NeuralNetwork의 인스턴스 instance를 생성하고 이를 devicefh 이동한 뒤, 구조(structure)를 출력합니다.

In [5]:
model =NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


모델을 사용하기 위해 입력 데이터를 전달합니다. 이는 일부 백그라운드 연산들과 함께 모델의 forward를 실행합니다. model.forward()를 직접 호출하지 마세요 !

모델에 입력을 전달하여 호출하면 2차원 텐서를 반환합니다. 2차원 텐서의 dim = 0 은 각 분류에 대한 원시 예측값 10개가, dim=1에는 각 출력의 개별 값들이 해당합니다. 원시 예측값을 nn.Softmax 모듈의 인스턴스에 통과시켜 예측 확률을 얻습니다.

In [7]:
X = torch.rand(1, 28, 28, device = device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([3])


# 모델 계층(Layer)
FashionMNIST 모델의 계층들을 살펴보겠습니다.
이를 설명하기 위해 28x28 크기의 이미지 3개로 구성된 minibatch를 가져와, 신경망을 통과할 때 어떤 일이 발생하는지 알아보겠습니다.

In [10]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


# nn.Flatten
nn.Flatten 계층을 초기화하여 각 28x28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 반환합니다.

In [11]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


# nn.Linear
선형 계층은 저장된 weight와 bias를 사용하여 입력에 선형 변환을 적용하는 모듈입니다.


In [12]:
layer1=nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


# nn.ReLU
비선형 활성화 activation은 모델의 입력과 출력 사이에 복잡한 관계를 만듭니다.
비선형 활성화는 선형 변환 후에 적용되어 비선형성을 도입하고, 신경망이 다양한 현상을 학습할 수 있도록 돕습니다.

이 모델에서는 nn.ReLU를 선형 계층들 사이에 사용하지만, 모델을 만들 때는 비선형성을 가진 다른 활성화를 도입할 수도 있습니다.

In [13]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.7378, -0.3603, -0.4851, -0.9009, -0.2597,  0.3477, -0.1873, -0.4886,
          0.2555,  0.4735,  0.2715, -0.4230, -0.0635,  0.2873,  0.2036, -0.1826,
          0.5243,  0.1479, -0.0523,  0.0627],
        [ 0.4657, -0.5421, -0.1602, -0.7428,  0.0052,  0.2343,  0.1540, -0.6729,
          0.3284,  0.4818, -0.0408, -0.2830, -0.2413,  0.2389, -0.1496,  0.0215,
          0.6174, -0.0575,  0.2622,  0.0609],
        [ 0.4628,  0.1141, -0.1774, -1.1723, -0.5661,  0.2526, -0.0135, -0.3357,
         -0.0679,  0.2704,  0.4977, -0.1836, -0.2930,  0.3616,  0.0696, -0.2027,
          0.4658,  0.0492,  0.0074, -0.1861]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.7378, 0.0000, 0.0000, 0.0000, 0.0000, 0.3477, 0.0000, 0.0000, 0.2555,
         0.4735, 0.2715, 0.0000, 0.0000, 0.2873, 0.2036, 0.0000, 0.5243, 0.1479,
         0.0000, 0.0627],
        [0.4657, 0.0000, 0.0000, 0.0000, 0.0052, 0.2343, 0.1540, 0.0000, 0.3284,
         0.4818, 0.0000, 0.0000, 0.0000, 0.2389, 0.00

# nn.Sequential
nn.Sequential은 순서를 갖는 모듈의 컨테이너 입니다. 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달됩니다.
순차 컨테이너(sequential container)를 사용하여 아래의 seq_modules와 같은 신경망을 빠르게 만들 수 있습니다.

In [16]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

# nn.Softmax
신경망의 마지막 선형 계층은 nn.Softmax 모듈에 전달될 ([-infty, infty] 범위의 원시 값(raw value)인) logits를 반환합니다. logits는 모델의 각 분류에 대한 예측 확률을 나타내도록 [0, 1] 범위로 비례하여 조정됩니다.
dim 매개변수는 값의 합이 1이 되는 차원을 나타냅니다.

In [17]:
softmax = nn.Softmax(dim = 1)
pred_probab = softmax(logits)


# 모델 매개변수
신경망 내부의 많은 계층들은 매개변수화 됩니다. 즉, 학습 중에 최적화되는 가중치와 편향과 연관지어집니다.  nn.Module을 상속하면 모델 객체 내부의 모든 필드들의이 자동으로 추적되며, 모델의 parameter()및 named_parameters() 메소드로 모든 매개변수에 접근할 수 있게 됩니다.

In [18]:
print(f"Model structure: {model}\n\n")
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]}")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0083, -0.0043,  0.0248,  ...,  0.0159,  0.0321,  0.0153],
        [-0.0337,  0.0024, -0.0226,  ...,  0.0258,  0.0101, -0.0287]],
       grad_fn=<SliceBackward0>)
Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([0.0211, 0.0263], grad_fn=<SliceBackward0>)
Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0404,  0.0050, -0.0335,  ..., -0.0311, -0.0245, -0.0223],
        [-0.0185, -0.0043, -0.0102,  ..., -0.0084, -0.0384, -0.0105]],
       grad_fn=<SliceBackward0>)
Layer: linear_relu_stack.2.bias | Size: torch